In [31]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(center=(39.3327459, -76.62337004), zoom=9)

dark_matter_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR)
m.add_layer(dark_matter_layer)
m

Map(center=[39.3327459, -76.62337004], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

In [32]:
m = Map?

Init signature: Map(*args, **kwargs)
Docstring:     
Map class.

The Map class is the main widget in ipyleaflet.

Attributes
----------
layers: list of Layer instances
    The list of layers that are currently on the map.
controls: list of Control instances
    The list of controls that are currently on the map.
center: list, default [0, 0]
    The current center of the map.
zoom: float, default 12
    The current zoom value of the map.
zoom_snap: float, default 1
    Forces the map’s zoom level to always be a multiple of this..
zoom_delta: float, default 1
    Controls how much the map’s zoom level will change after
    pressing + or - on the keyboard, or using the zoom controls.
crs: projection, default projections.EPSG3857
    Coordinate reference system, which can be ‘Earth’, ‘EPSG3395’, ‘EPSG3857’,
    ‘EPSG4326’, ‘Base’, ‘Simple’ or user defined projection.
Init docstring: Public constructor
File:           ~/.venv/astro_disco/lib/python3.8/site-packages/ipyleaflet/leaflet.py
Typ

In [ ]:
m = Map